In [11]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize
from nltk.tokenize import RegexpTokenizer
import itertools

# HW2:  Модель Word2vec
### Выполнили:  

* Булгаков Дмитрий

* Тефикова Алие

### Группа ИАД-2

## 0. Loading data

Составьте достаточно большую коллекцию текстов на любом языке или ис-пользуйте тексты отсюда:<br>
https://www.kaggle.com/c/word2vec-nlp-tutorial/data.

In [12]:
text_dataset = pd.read_csv('data/unlabeledTrainData.csv', sep=';', encoding = 'latin1')

In [13]:
text_dataset.head(n=5)

,id,review
0,9999_0,"Watching Time Chasers, it obvious that it was ..."
1,45057_0,I saw this film about 20 years ago and remembe...
2,15561_0,"Minor SpoilersIn New York, Joan Barnard (Elvir..."
3,7161_0,I went to see this film with a great deal of e...
4,43971_0,"Yes, I agree with everyone on this site this m..."


In [14]:
def split_dataset_into_list_sentences(df, size = None):
    df_sentences_list = []
    if (size == None):
        size = len(df)
    for i in range(0, size):
        df_sentences_list.append(sent_tokenize(df.review.values[i]))
    return df_sentences_list

In [15]:
def flatten_list_of_lists(list_of_lists):
    return list(itertools.chain.from_iterable(list_of_lists))

In [16]:
def split_sentence_into_words_list(sentence):
    tokenizer = RegexpTokenizer(r'\w+')
    return [x.lower() for x in tokenizer.tokenize(sentence)]

In [17]:
def split_list_of_sentences_into_words_list(list_sentences):
    list_sentences_words = []
    for sentence in list_sentences:
        list_sentences_words.append(split_sentence_into_words_list(sentence))
    return list_sentences_words

In [18]:
def process_dataset(dataset):
    list_sentences = split_dataset_into_list_sentences(dataset)
    list_sentences = flatten_list_of_lists(list_sentences)
    list_sentences_words = split_list_of_sentences_into_words_list(list_sentences)
    return list_sentences_words

In [19]:
%%time
processed_dataset = process_dataset(text_dataset)

CPU times: user 52.8 s, sys: 2.19 s, total: 55 s
Wall time: 1min 1s


## 1. Fitting Word2Vec model

Обучите модель word2vec. Оцените время обучения модели,используя модуль time. Есть два варианта обучения модели: по отзывам целиком и с уче-там границ предложений. В принципе, погрешностью, которая возникает в первом случае можно пренебречь, но если вы хотите учитывать границы предложений, то можно использовать sent_tokenize из nltk.tokenize

In [20]:
saved_model_name = 'savedmodel'

As Word2Vec tooks a lot of time to fit model, saving it to file

In [21]:
%%time
# model = Word2Vec(processed_dataset, workers=4)
# model.save(saved_model_name)

CPU times: user 3min 4s, sys: 2.54 s, total: 3min 6s
Wall time: 1min 16s


In [22]:
model = Word2Vec.load(saved_model_name)

## 2. Looking for similar words

Примеры практически безошибочного (во всяком случае для ТОП-3) определения синонимов с помощью most_similar для следующих слов: 

**Amazing**

In [23]:
model.most_similar(positive = ['amazing'], topn = 3)

[('incredible', 0.8616175651550293),
 ('awesome', 0.832375168800354),
 ('outstanding', 0.7665010094642639)]

**Stunning**

In [24]:
model.most_similar(positive = ['stunning'], topn = 3)

[('breathtaking', 0.8776392936706543),
 ('magnificent', 0.8123955130577087),
 ('dazzling', 0.8080258369445801)]

**Great**

In [25]:
model.most_similar(positive = ['great'], topn = 3)

[('wonderful', 0.8272826075553894),
 ('fantastic', 0.8226152062416077),
 ('terrific', 0.7967977523803711)]

**Thrilling**

In [26]:
model.most_similar(positive = ['thrilling'], topn = 3)

[('suspenseful', 0.8708139061927795),
 ('gripping', 0.85813307762146),
 ('exciting', 0.8081084489822388)]

**Huge**

In [27]:
model.most_similar(positive = ['huge'], topn = 3)

[('big', 0.7841212749481201),
 ('massive', 0.759773850440979),
 ('large', 0.6307076215744019)]

**Recommendation**

In [28]:
model.most_similar(positive = ['recommendation'], topn = 3)

[('advice', 0.758362352848053),
 ('estimation', 0.7475383877754211),
 ('suggestion', 0.7331488728523254)]

Примеры, где синонимами являются только ТОП-**2** похожих слова: 

**Film**

In [29]:
model.most_similar(positive = ['film'], topn = 3)

[('movie', 0.946456789970398),
 ('picture', 0.7363325357437134),
 ('documentary', 0.7319446802139282)]

**Channel**

In [30]:
model.most_similar(positive = ['channel'], topn = 3)

[('network', 0.8229336738586426),
 ('cable', 0.8031548857688904),
 ('hbo', 0.7376188635826111)]

**Place**

In [31]:
model.most_similar(positive = ['place'], topn = 3)

[('backseat', 0.5451779365539551),
 ('area', 0.536838710308075),
 ('advantage', 0.5127078294754028)]

Как видно, и documentry, и bbc, и country является похожими по смыслу словами, но не синонимами, так как это скорее слова, которые включается в большое множество film, channel и place, т.е. являются их подмножествами, а не эквивалентны им.

**Kind**

In [32]:
model.most_similar(positive = ['kind'], topn = 3)

[('sort', 0.8997496962547302),
 ('type', 0.7444095611572266),
 ('semblance', 0.5400596261024475)]

В случае с kind, опять же синонимами являются только первые два слова, что также видно и по значению ***похожести????*** или как сказать

**Lucky**

In [33]:
model.most_similar(positive = ['lucky'], topn = 3)

[('fortunate', 0.7112295031547546),
 ('grateful', 0.5837315320968628),
 ('eager', 0.5461816191673279)]

В данном примере синонимом является уже только первое слово, что говорит о том, что most_similar не всегда подходит для определения похожих слов с точки зрения их лексического значения. Также это будет подтверждено следующими примерами.

Случаи, когда самым схожим словом является - **антоним** (love - hate, day - night):

**Love**

In [34]:
model.most_similar(positive = ['love'], topn = 3)

[('hate', 0.5882089138031006),
 ('bermuda', 0.5565627813339233),
 ('loved', 0.5049281120300293)]

**Day**

In [35]:
model.most_similar(positive = ['day'], topn = 3)

[('week', 0.6401285529136658),
 ('night', 0.6350167393684387),
 ('morning', 0.5970383882522583)]

Можно заметить, что большую часть наших удачных примеров составляют прилагательные. Это связано с тем, что именно для прилагательных модель выдает лучшие результаты, то есть точнее определяет синонимы. Скорее всего, это связано с тем, что при написании рецензий используется огромное количество всевозможных прилагательных, что дает модели возможность лучше обучиться. Благодаря этому, для данного текста most_similar действительно подходит для определения синонимов слов, точность результатов которого значительно повышается, если часть речи проверяемого слова - прилагательное.  

## 3. Looking for associations

Приведите 5-10 примеров использования .most_similar для определения ассоциаций (А к Б, как В к?). Корректно ли найдены ассоциации?

In [36]:
model.most_similar(positive = ['hot'], negative = ['weather'], topn = 3)

[('cute', 0.5197432637214661),
 ('sexy', 0.5062525272369385),
 ('adorable', 0.4691374599933624)]

In [37]:
model.most_similar(positive = ['america'], topn = 3)

[('japan', 0.7715083956718445),
 ('china', 0.7648563385009766),
 ('korea', 0.7630176544189453)]

In [38]:
model.most_similar(positive = ['car', 'accident'], topn = 3)

[('plane', 0.7602980136871338),
 ('helicopter', 0.7549397945404053),
 ('truck', 0.7372640371322632)]

In [39]:
model.most_similar(positive = ['mother', 'father'], topn = 3)

[('husband', 0.8845164179801941),
 ('daughter', 0.8816182017326355),
 ('son', 0.8680322170257568)]

In [40]:
model.most_similar(positive = ['work', 'job'], negative = ['education'], topn = 3)

[('performance', 0.500205934047699),
 ('direction', 0.44860243797302246),
 ('screenplay', 0.42101961374282837)]

In [41]:
model.most_similar(positive = ['war', 'death', 'soldiers'], topn = 3)

[('vietnam', 0.7079989910125732),
 ('nazi', 0.7017005681991577),
 ('iraq', 0.7010694146156311)]

In [42]:
model.most_similar(positive = ['crime', 'blood', 'victim'], topn = 3)

[('maniac', 0.6700836420059204),
 ('rage', 0.6627383232116699),
 ('murder', 0.6604952216148376)]

In [43]:
model.most_similar(positive = ['film', 'funny'], topn = 3)

[('movie', 0.8122972249984741),
 ('entertaining', 0.6669540405273438),
 ('it', 0.665064811706543)]

В целом, можно отметить, что модель хорошо обучилась, и хорошо определяет сложные ассоциации, делая это с высокой степенью качества. 

## 4. Looking for inappropriate words

Приведите 5-10 примеров использования .doesnt_match для определения лишнего слова. Корректно ли найдены лишние слова?

Model finds words that out of context

In [44]:
model.doesnt_match("pizza food popcorn salad fries character".split())

'character'

In [45]:
model.doesnt_match("movie film cinema hero actor singer".split())

'singer'

In [46]:
model.doesnt_match("schindlers jews fun pain prison kill".split())

'schindlers'

In [47]:
model.doesnt_match("love joy like enjoy excite boring ".split())

'boring'

And even distinguishes male from female names

In [48]:
model.doesnt_match("mary joan caroline john malisa keith".split())

'john'

Как и в предыдущих случаях, модель отработала на ура:) В результате использования метода .doesnt_match удается не только выделять слова, не подходящие по контексту, но и выделять более сложные зависимости. Такие, как, например, дифференциация женских имен от мужских.

## 5. Checking commutativity rule

Попробуйте найти такие пары и тройки слов, для которых:
* не выполняются свойства коммутативности и транзитивности относительно операции определения близких слов.
* выполняются свойства коммутативности и транзитивности отно-сительно операции определения близких слов.

Обозначим отношение “входить в топ-3 по.most_similar” символом ○.<br>
**Коммутативность:** x ○ y => y ○ x <br>
**Транзитивность:** x ○ y, y ○ z => x ○ z

Пара слов, для которых не выполняется свойство коммутативности

In [49]:
model.most_similar(positive = ['groove'], topn = 3)

[('saxons', 0.692660927772522),
 ('fingertips', 0.6818060874938965),
 ('mounties', 0.6711975336074829)]

In [50]:
model.most_similar(positive = ['hurst'], topn = 3)

[('fannie', 0.885786235332489),
 ('hubert', 0.8662824630737305),
 ('pallenberg', 0.8519117832183838)]

Как видно, Hurst входит в ТОП-3 похожих слов для Groove, а вот Groove не в ходит в ТОП-3 Hurst, зато в него входят имена, из чего делаем вывод, что в контексте наших данных Hurst встречалось чаще всего как фамилия, нежели как слово "бугор". 

Пара слов, для которых выполняется свойство коммутативности

In [51]:
model.most_similar(positive = ['lovely'], topn = 3)

[('gorgeous', 0.8244462013244629),
 ('beautiful', 0.7893744707107544),
 ('sexy', 0.6904053688049316)]

In [52]:
model.most_similar(positive = ['gorgeous'], topn = 3)

[('beautiful', 0.843631386756897),
 ('lovely', 0.8244462013244629),
 ('stunning', 0.8003969192504883)]

Как уже было замечено ранее, при написании рецензий авторы чаще всего используют прилагательные, поэтому эта часть речи, а особенно наиболее популярные слова в больших случаях обладает свойством коммутативности, что подтверждается нашим примером с lovely и gorgeous.

Проверим транзитивность: случай, когда свойство выполняется:

In [53]:
model.most_similar(positive = ['film'], topn = 3)

[('movie', 0.946456789970398),
 ('picture', 0.7363325357437134),
 ('documentary', 0.7319446802139282)]

In [54]:
model.most_similar(positive = ['movie'], topn = 3)

[('film', 0.946456789970398),
 ('flick', 0.7504115700721741),
 ('picture', 0.656548023223877)]

In [55]:
model.most_similar(positive = ['flick'], topn = 3)

[('movie', 0.7504115700721741),
 ('thriller', 0.7113186717033386),
 ('film', 0.7006672620773315)]

В данном случае выполняются свойства коммутативности и транзитивности film, movie коммутативны, а film и flick транзитивны

И, наконец, случай, когда свойство транзитивности не выполняется:

In [56]:
model.most_similar(positive = ['cat'], topn = 3)

[('sheep', 0.767533540725708),
 ('dog', 0.7201465368270874),
 ('tiger', 0.70553058385849)]

In [57]:
model.most_similar(positive = ['dog'], topn = 3)

[('cat', 0.720146656036377),
 ('demon', 0.6932040452957153),
 ('horse', 0.6806365847587585)]

In [58]:
model.most_similar(positive = ['demon'], topn = 3)

[('robot', 0.8184897899627686),
 ('maniac', 0.7666633725166321),
 ('rat', 0.7578243017196655)]

В этом же случае коммутативность между cat и dog присутствует,но вот транзитивность между cat и demon уже не наблюдается.